In [1]:
import pandas as pd
import requests
import json

## Enter your API keys from Moralis

In [49]:
headers = {
    'accept': 'application/json',
    'X-API-Key': 'pkhgpCPTU5AFiwXHrTwNCVXBIPfS3HZBLpOp1YOc8SOKxA8dUSVCEt9sg8hFjokU',
}

## Enter the Contract address that you want to explore

In [50]:
contract_addr = "0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb"

## To get All Owners of the NFT

In [53]:
params = {
    'chain': 'eth',
    'format': 'decimal',
}
url = f'https://deep-index.moralis.io/api/v2/nft/{contract_addr}/owners'

holder_list = requests.get(url, headers=headers, params=params)

In [56]:
with open('Holder_address.json', 'w') as outfile:
    outfile.write(holder_list.text)

## Get the Balance in the wallet of each owners

In [57]:
params = {
    'chain': 'eth',
}

In [58]:
owner_list = []
owner_result = []
for result in json.loads(holder_list.text)["result"]:
    if(len(owner_list)<20):
        owner_list.append(result["owner_of"])
    else:
        owner_addr = ",".join(owner_list)
        url = f"https://api.etherscan.io/api?module=account&action=balancemulti&address={owner_addr}&tag=latest&apikey=57GBAHGQSE3CMYNZQY81V1N32YJZNCCUNZ"
        response = requests.get(url, verify=True)
        result = json.loads(response.text)["result"]
        owner_result.extend(result)
        owner_list = []

In [59]:
pd_owner_result = pd.DataFrame(owner_result)
owner_value_count = pd_owner_result.value_counts(subset="account").to_frame().reset_index()
owner_value_count.columns = ["account", "counts"]
pd_unique_owner = pd_owner_result.drop_duplicates(ignore_index=True)

pd_owner = pd_unique_owner.merge(owner_value_count, on="account")
pd_owner["balance"] = pd_owner["balance"].astype(float)/1e18

# pd_owner.sort_values(by="counts")
# pd_owner.sort_values(by="balance")

In [70]:
owner_json = pd_owner.to_json(orient="records")
with open('Holders_balance.json', 'w') as outfile:
    outfile.write(owner_json)

## Count Holders with balance under threshold

In [71]:
threshold = 1
under_count = pd_owner[pd_owner["balance"]<=threshold]
under_count

,account,balance,counts
1,0x2aaba4b94a805f0fea696dc11de324c2320ff4e1,0.000000,1
4,0xb7f7f6c52f2e2fdb1963eab30438024864c313f6,0.000000,19
6,0x8b6fede39b04c23bfcc7fe74e2680139a68bdbd7,0.310470,1
7,0xa9dc1bf6a46bc0d03c2b3d621d434dd6e998190d,0.991008,1
9,0xbcd7e572be79a7aa288c35ff6b624da447710107,0.073051,1
...,...,...,...
185,0xa858ddc0445d8131dac4d1de01f834ffcba52ef1,0.022700,116
186,0x27f8cf716723daa1dd131f5013fe5352052919b7,0.488651,1
191,0x23d23d05eca973294c648be9db31e6e6ae90c0c7,0.165433,1
192,0x629d33cc77b19f479218f10752a8aa1b70f0b89a,0.000000,1


In [72]:
threshold = 200
over_count = pd_owner[pd_owner["balance"]>threshold]
over_count

,account,balance,counts
3,0x1919db36ca2fa2e15f9000fd9cdc2edcf863e685,244.941056,6
15,0x96e1199931a9e201a4d209262c8c379e2c0e4852,554.599304,1
17,0x2503e93fbb53fe9821630746a4070bec2f7d496d,462.026515,1
80,0xbb26a6da4d918682f58cc91bd3fb251dd28549d2,410.389711,10
85,0x484c6dcd50978b9d525cc20a02d5801f492931f8,219.274267,2
103,0x534a0076fb7c2b1f83fa21497429ad7ad3bd7587,280.013514,1
105,0xcb9cadd85c510ff5a4fb2d2bb6fb4b74ff3aee34,332.431470,1
111,0x7a63bf9561529a64ec8b708b9930dff88e874b5f,252.000000,1
147,0x1cf2b8c64aed32bff2ae80e701681316d3212afd,218.669745,1
162,0x8bbc693d042cea740e4ff01d7e0efb36110c36bf,300.198435,1


## Get all NFT's in the interest wallet

In [73]:
params = {
    'chain': 'eth',
    'format': 'decimal',
}

In [74]:
interest_address = over_count["account"]
NFT_result_list = []
mapping = {}

""" Get all token of one account """
i = 0
for owner_addr in interest_address:
    url = f"https://deep-index.moralis.io/api/v2/{owner_addr}/nft"
    response = json.loads(requests.get(url, headers=headers, params=params).text)["result"]
    
    for r in response:
        if r["token_address"] in mapping.keys():
            NFT_result_list[mapping[r["token_address"]]]["Numbers"] += 1
        else:
            mapping[r["token_address"]] = i
            i += 1
            tmp = {
                'token_address': r["token_address"],
                'name': r["name"],
                'symbol': r["symbol"],
                'Numbers': 1,
            }
            NFT_result_list.append(tmp)

In [75]:
# Sort by collection Numbers
NFT_result_list = sorted(NFT_result_list, key=lambda r: r["Numbers"], reverse=True)
NFT_result_list

[{'token_address': '0xc799f57355677a19b91c722734743215513dece5',
  'name': 'ShatteredEon',
  'symbol': 'Colonists',
  'Numbers': 100},
 {'token_address': '0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb',
  'name': 'CRYPTOPUNKS',
  'symbol': 'Ͼ',
  'Numbers': 68},
 {'token_address': '0x4b3406a41399c7fd2ba65cbc93697ad9e7ea61e5',
  'name': 'Lost Poets',
  'symbol': 'POETS',
  'Numbers': 64},
 {'token_address': '0x495f947276749ce646f68ac8c248420045cb7b5e',
  'name': 'OpenSea Shared Storefront',
  'symbol': 'OPENSTORE',
  'Numbers': 63},
 {'token_address': '0xba30e5f9bb24caa003e9f2f0497ad287fdf95623',
  'name': 'BoredApeKennelClub',
  'symbol': 'BAKC',
  'Numbers': 57},
 {'token_address': '0xaf90d15098275db315979b00f8a308c8c0bb980f',
  'name': 'Gutter Punks Flyer - CryptoPunks',
  'symbol': 'GPFCP',
  'Numbers': 54},
 {'token_address': '0x60e4d786628fea6478f785a6d7e704777c86a7c6',
  'name': 'MutantApeYachtClub',
  'symbol': 'MAYC',
  'Numbers': 54},
 {'token_address': '0x3b3ee1931dc30c1957379fa

## Add Lowest Prices to list

In [76]:
price_params = {
    'chain': 'eth',
    'marketplace': 'opensea',
    'days': 15
}

In [77]:
for r in NFT_result_list:
    price_url = f"https://deep-index.moralis.io/api/v2/nft/{r['token_address']}/lowestprice"
    price_response = requests.get(price_url, headers=headers, params=price_params)
    if price_response.status_code == 200:
        price_response = json.loads(price_response.text)
        price = float(price_response["price"])/ 1e18
        r['price'] = price
    else:
        r['price'] = 0

In [78]:
# Sort by value
NFT_result_list = sorted(NFT_result_list, key = lambda r: r['Numbers']*r['price'], reverse = True)
NFT_result_list

[{'token_address': '0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d',
  'name': 'BoredApeYachtClub',
  'symbol': 'BAYC',
  'Numbers': 32,
  'price': 96.5},
 {'token_address': '0xd4e4078ca3495de5b1d4db434bebc5a986197782',
  'name': 'Autoglyphs',
  'symbol': '☵',
  'Numbers': 1,
  'price': 172.0},
 {'token_address': '0x9c8ff314c9bc7f6e59a9d9225fb22946427edc03',
  'name': 'Nouns',
  'symbol': 'NOUN',
  'Numbers': 2,
  'price': 60.0},
 {'token_address': '0x49cf6f5d44e70224e2e23fdcdd2c053f30ada28b',
  'name': 'CloneX',
  'symbol': 'CloneX',
  'Numbers': 8,
  'price': 13.0},
 {'token_address': '0x7bd29408f11d2bfc23c34f18275bbf23bb716bc7',
  'name': 'Meebits',
  'symbol': '⚇',
  'Numbers': 26,
  'price': 2.6959},
 {'token_address': '0xe785e82358879f061bc3dcac6f0444462d4b5330',
  'name': 'World Of Women',
  'symbol': 'WOW',
  'Numbers': 5,
  'price': 7.7},
 {'token_address': '0x8479277aacff4663aa4241085a7e27934a0b0840',
  'name': 'Realms of Ether',
  'symbol': 'REALM',
  'Numbers': 5,
  'price': 6.

In [79]:
with open('NFT_TokenList.json', 'w') as outfile:
    outfile.write(json.dumps(NFT_result_list))

## Get all ERC20 tokens in interest wallet

In [82]:
params = {
    'chain': 'eth',
}

In [90]:
interest_address = over_count["account"]
erc20_result_list = []
mapping = {}

""" Get all token of one account """
i = 0
for owner_addr in interest_address:
    url = f"https://deep-index.moralis.io/api/v2/{owner_addr}/erc20"
    response = json.loads(requests.get(url, headers=headers, params=params).text)
    
    for r in response:
        decimal = float(r["decimals"]) if r["decimals"] else 1e-18
        if r["token_address"] in mapping.keys():
            erc20_result_list[mapping[r["token_address"]]]["Numbers"] += float(r["balance"])/ 10**decimal
        else:
            mapping[r["token_address"]] = i
            i += 1
            tmp = {
                'token_address': r["token_address"],
                'name': r["name"],
                'symbol': r["symbol"],
                'Numbers': float(r["balance"])/ 10**decimal,
            }
            erc20_result_list.append(tmp)

In [91]:
# Sort by collection Numbers
erc20_result_list = sorted(erc20_result_list, key=lambda r: r["Numbers"], reverse=True)
erc20_result_list

[{'token_address': '0x4b3464fe6a37ce9cfe2956056f447261fd7ea614',
  'name': 'Rabbit Rocket',
  'symbol': 'Rabbit',
  'Numbers': 44758912878009.23},
 {'token_address': '0xa3986e46f051e275e575930107848c4eba83a788',
  'name': 'Squid Game V2',
  'symbol': 'Squid Game V2',
  'Numbers': 13907497505997.0},
 {'token_address': '0xf27adb015958b4f02128ecfa3ce59b3772e9676a',
  'name': 'Harvard Metaverse',
  'symbol': 'Harvard',
  'Numbers': 2116467992977.0798},
 {'token_address': '0xce4fd7ea4c894ed145ecb996a79c336f3a279334',
  'name': 'Mfers Coin',
  'symbol': 'Mfers',
  'Numbers': 1825479101141.389},
 {'token_address': '0xf71ba4058c7ea23992a2311add6862d854b3d229',
  'name': 'MAYC COIN',
  'symbol': 'MAYC',
  'Numbers': 1708211504684.831},
 {'token_address': '0xd3bf5b01aed2546645708ccbfc1ac0753089eaec',
  'name': 'Nikeland Metaverse',
  'symbol': 'Nikeland',
  'Numbers': 1602384558800.8508},
 {'token_address': '0x9fc499e6215f2aa51b61eae1cfffdf52418bf712',
  'name': 'Disney Metaverse',
  'symbol': '

## Add Liquidity Prices to list

In [92]:
""" Lowest Prices"""
price_params = {
    'chain': 'eth',
}

In [93]:
for r in erc20_result_list:
    price_url = f"https://deep-index.moralis.io/api/v2/erc20/{r['token_address']}/price"
    price_response = requests.get(price_url, headers=headers, params=price_params)
    if price_response.status_code == 200:
        price_response = json.loads(price_response.text)
        r['price'] = float(price_response["usdPrice"])
    else:
        r['price'] = 0

In [94]:
# Sort by value
erc20_result_list = sorted(erc20_result_list, key = lambda r: r['Numbers']*r['price'], reverse = True)
erc20_result_list

[{'token_address': '0x4b3464fe6a37ce9cfe2956056f447261fd7ea614',
  'name': 'Rabbit Rocket',
  'symbol': 'Rabbit',
  'Numbers': 44758912878009.23,
  'price': 1.38776508984949e+20},
 {'token_address': '0xa3986e46f051e275e575930107848c4eba83a788',
  'name': 'Squid Game V2',
  'symbol': 'Squid Game V2',
  'Numbers': 13907497505997.0,
  'price': 3.469412724623725e+20},
 {'token_address': '0x14026711b9b89f699916f608fa997944e3269f20',
  'name': 'Monkeyverse',
  'symbol': 'MONKE',
  'Numbers': 331362.2029020689,
  'price': 1.1548903326567561e+19},
 {'token_address': '0x9b4109cfbf22348260db8d36780554d7fd3898ff',
  'name': 'Subway Metaverse',
  'symbol': 'Subway',
  'Numbers': 6077755441.46719,
  'price': 0.1999051721508243},
 {'token_address': '0x9fc499e6215f2aa51b61eae1cfffdf52418bf712',
  'name': 'Disney Metaverse',
  'symbol': 'DISNEY',
  'Numbers': 1579537761076.001,
  'price': 0.000215076439955088},
 {'token_address': '0xf71ba4058c7ea23992a2311add6862d854b3d229',
  'name': 'MAYC COIN',
  '

In [95]:
with open('ERC20_TokenList.json', 'w') as outfile:
    outfile.write(json.dumps(erc20_result_list))

In [ ]:
erc20_result_list

In [ ]:
# url = f"https://api.ethplorer.io/getAddressInfo/{addr}?apiKey=freekey"

# response = requests.get(url, verify=True)
# print(response.text["result"])

# https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0x9e1c80662d04b2c94dce0a46e5b1f5b48ce97e26&address=0x95ebe7d8089d4c8bcee0bde19c9e867cbeeefd00&tag=latest&apikey=57GBAHGQSE3CMYNZQY81V1N32YJZNCCUNZ
# url = "https://api.etherscan.io/api?module=account&action=balancemulti&address=0xddbd2b932c763ba5b1b7ae3b362eac3e8d40121a,0x63a9975ba31b0b9626b34300f7f627147df1f526,0x198ef1ec325a96cc354c7266a038be8b5c558f67&tag=latest&apikey=57GBAHGQSE3CMYNZQY81V1N32YJZNCCUNZ"